In [4]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import datetime

# Step2: HTML解析して株価データを抽出する関数
def extract_stock_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 必要なデータを抽出（仮: 適切なCSSセレクタを確認して修正）
    date = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(1)").get_text(strip=True)  # 日付
    open_price = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(2)").get_text(strip=True).replace(',', '') # 始値
    high_price = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(3)").get_text(strip=True).replace(',', '') # 高値
    low_price = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(4)").get_text(strip=True).replace(',', '') # 安値
    close_price = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(5)").get_text(strip=True).replace(',', '') # 終値

    return [date, open_price, high_price, low_price, close_price]

# Step3: Webページからデータを取得する関数
def get_stock_values(driver, url):
    driver.get(url)
    time.sleep(20)  # 広告が消えるまで待機
    graph_xy = driver.find_elements(By.CSS_SELECTOR, '.highcharts-grid')[1]
    #driver.execute_script("arguments[0].scrollIntoView(true);", graph_xy) # グラフを画面内に収める
    #time.sleep(1)  # スクロール時間の確保

    # グラフ要素の幅を取得する
    graph_width = graph_xy.rect['width']

    # グラフの右端にマウスポインタを移動する
    actions = ActionChains(driver)
    actions.move_to_element(graph_xy).perform()
    time.sleep(0.2)  # カーソル移動時間の確保

    actions.move_by_offset(graph_width // 2, 0).perform()
    time.sleep(0.2)  # カーソル移動時間の確保

    # グラフ幅分だけ移動してデータを取得
    data_points = []
    step = 5 #1日は5～6pxで構成されている

    for x_offset in range(graph_width, 0, -step):  # 右端から左端へ
        #actions.move_to_element_with_offset(graph_xy, x_offset, 0).perform()

        # HTML取得
        html = driver.page_source
        stock_data = extract_stock_data(html)
        data_points.append(stock_data)
        # カーソルの移動
        if x_offset >= step:
            actions.move_by_offset(-step, 0).perform()
            time.sleep(0.2)  # カーソル移動時間の確保


    return data_points

# Step4: mainコード
if __name__ == "__main__":
    # スクレイピングの開始時間
    start_time = time.time()
    
    

    # Selenium WebDriverのセットアップ
    driver = webdriver.Chrome()

    try:
        url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"
        stock_values = get_stock_values(driver, url)
        stock_values_sort = sorted(set(map(tuple, stock_values)), key=lambda x: stock_values.index(list(x)))

        # スクレイピングの終了時間
        end_time = time.time()
        elapsed_time = end_time - start_time

        # 結果の表示
        print(f"スクレイピング時間: {elapsed_time:.2f}秒")
        print("日付, 始値, 高値, 安値, 終値")
        for stock in stock_values_sort:
            print(", ".join(map(str, stock)))

    finally:
        driver.quit()


スクレイピング時間: 98.36秒
日付, 始値, 高値, 安値, 終値
2025/1/28, 始値: 39400.02, 高値: 39448.55, 安値: 38886.05, 終値: 39016.87
2025/1/27, 始値: 40127.74, 高値: 40255.68, 安値: 39520.79, 終値: 39565.8
2025/1/24, 始値: 40060.49, 高値: 40279.79, 安値: 39806.69, 終値: 39931.98
2025/1/23, 始値: 39810.06, 高値: 40036.07, 安値: 39677.22, 終値: 39958.87
2025/1/22, 始値: 39355.22, 高値: 39694.57, 安値: 39332.63, 終値: 39646.25
2025/1/21, 始値: 39163.53, 高値: 39238.21, 安値: 38643.84, 終値: 39027.98
2025/1/20, 始値: 38671.77, 高値: 39032.93, 安値: 38671.77, 終値: 38902.5
2025/1/17, 始値: 38454.1, 高値: 38503.94, 安値: 38055.68, 終値: 38451.46
2025/1/16, 始値: 38732.65, 高値: 38932.54, 安値: 38426.2, 終値: 38572.6
2025/1/15, 始値: 38721.61, 高値: 38774.99, 安値: 38316.01, 終値: 38444.58
2025/1/14, 始値: 39010.96, 高値: 39054.35, 安値: 38305.91, 終値: 38474.3
2025/1/10, 始値: 39550.25, 高値: 39591.46, 安値: 39166.05, 終値: 39190.4
2025/1/9, 始値: 39888.91, 高値: 39930.07, 安値: 39385.05, 終値: 39605.09
2025/1/8, 始値: 39879.36, 高値: 40105.72, 安値: 39705.81, 終値: 39981.06
2025/1/7, 始値: 39584.36, 高値: 40288.8, 安値: 39584.3